In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm


fna_dir = '/mnt/raid7/Dachuang/Achuan/03_phage_host/db/host'
bowtie_dir = '/mnt/raid7/Dachuang/Achuan/03_phage_host/01_CRISPR/04_bowtie_virus'
output_dir = '/mnt/raid7/Dachuang/Achuan/03_phage_host/01_CRISPR/06_calculate_filtered'

parse_dir = '/mnt/raid7/Dachuang/Achuan/03_phage_host/01_CRISPR/05_parse_sam'
virus_json=json.load(open(f'/mnt/raid7/Dachuang/Achuan/03_phage_host/db/Virus_seqlen.json'))

virus_host_dict = json.load(
    open(f'/mnt/raid7/Dachuang/Achuan/03_phage_host/db/gold_virus_info.json'))
virus_all_ids=list(virus_host_dict.keys())

def mkdir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)
mkdir(output_dir)

## 解析sam文件

In [2]:

# mkdir(parse_dir)
# files = os.listdir(f"{bowtie_dir}")



# def parse_sam(filepath,filename):
#     contents=[]
#     with open(filepath, 'r') as f:
#         for line in f:
#             if line.startswith('@'):
#                 continue
#             else:
#                 tab=line.strip().split('\t')
#                 if tab[1] == '4':
#                     continue
#                 # print(tab[0], tab[2], tab[1], tab[5], tab[9])
#                 contents.append(
#                     f'{tab[0]}\t{tab[2]}\t{tab[1]}\t{tab[5]}\t{tab[9]}\n')
#     if contents:
#         with open(f"{parse_dir}/{filename}", 'w') as f:
#             for line in contents:
#                 f.write(line)


# for filename in tqdm(files):
#     parse_sam(f'{bowtie_dir }/{filename}/{filename}.sam', filename)


In [3]:
# from Bio.Seq import Seq
# my_seq = Seq('TTCATAGCAGAATCCACGTTTTTTATCCCAAATCTAAAAA')
# my_seq.reverse_complement()

## 计算spacer 比对分数

In [4]:
import math
import re
def calculate_spacer(filename):
    """
    calculate the evevry spacer score
    """
    from Bio import SeqIO
    from pandas.errors import EmptyDataError
    # read blastn file
    try:
        df = pd.read_table(
            f"{parse_dir}/{filename}", sep="\t", header=None)
    except EmptyDataError:
        return

    # add blastn output info
    df.columns = ["query", "subject", "flag","cigar", "spacer"]

    # 先统计指标：alignment_length*identity
    calculate_dict = {}
    for i in range(len(df)):
        # 防止之前不完整的id在列表中
        virus_id = df.iloc[i]['subject'].split('.')[0]
        if virus_id not in virus_all_ids:
            continue
        key = df.iloc[i]['query']+'__'+df.iloc[i]['subject']
        mismatch_list=re.findall(r"(\d+)[^M\d]", str(df.iloc[i]['cigar']))
        mismatch = sum(list(map(int, mismatch_list)))
        # 筛选mismatch <<1的
        spacer_length=len(df.iloc[i]['spacer'])
        if mismatch >1 or 1-mismatch/spacer_length <= 0.95:
            continue
        if key not in calculate_dict:
            calculate_dict[key] = {'single': []}
        calculate_dict[key]['single'].append(1-mismatch/spacer_length)



    for key in calculate_dict:
        prophage_id, virus_id = key.split('__')
        virus_length = virus_json[virus_id]
        """coverage因为prophage length可能比virus length大，不能说明好坏,所以取消这个指标
        prophage_length = len(pro_fasta_dict[prophage_id].seq)
        coverage = prophage_length/virus_length
        calculate_dict[key]['score'] = sum(
            calculate_dict[key]['single'])/virus_length*coverage
        """
        duplicate_number = len(calculate_dict[key]['single'])
        calculate_dict[key]['score'] = np.max(
            calculate_dict[key]['single'])*(duplicate_number*0.1+1)


    # calculate_dict to table
    sorted_dict = dict(sorted(calculate_dict.items(),
                    key=lambda x: x[1]['score'], reverse=True))
    with open(f'{output_dir}/{filename}.tsv', 'w') as f:
        for key in sorted_dict:
            prophage_id, virus_id = key.split('__')
            score = sorted_dict[key]['score']
            f.write(f'{filename}\t{virus_id}\t{score}\n')
def sigmoid(x):
    return 1/(1+math.exp(-x))


In [5]:
!rm /mnt/raid7/Dachuang/Achuan/03_phage_host/01_CRISPR/06_calculate_filtered/*

In [6]:
files = os.listdir(f"{parse_dir}")
for filename in tqdm(files):
    calculate_spacer(filename)



100%|██████████| 73/73 [00:00<00:00, 86.76it/s]


## 合并所有文件


In [7]:
!mkdir -p /mnt/raid7/Dachuang/Achuan/03_phage_host/01_CRISPR/07_score
! cat /mnt/raid7/Dachuang/Achuan/03_phage_host/01_CRISPR/06_calculate_filtered/*.tsv > /mnt/raid7/Dachuang/Achuan/03_phage_host/01_CRISPR/07_score/all_score_filtered.tsv

## 解析all_score.tsv

- task1： 获取每个virus-host 的 prophage score，对于每个virus找把分数最高的host当做是病毒的host
- task2： 给每个virus-host 分配标签，1代表正确，0代表错误

In [8]:
score_dir='/mnt/raid7/Dachuang/Achuan/03_phage_host/01_CRISPR/07_score/'

In [9]:
import pandas as pd
df_score=pd.read_table(
    f'{score_dir}/all_score_filtered.tsv', header=None)
df_score.columns = ['host', 'virus', 'score']

df_score


,host,virus,score
0,GCF_000007005.1,NC_023585.1,2.3
1,GCF_000007005.1,NC_034620.1,1.4
2,GCF_000007005.1,NC_034625.1,1.2
3,GCF_000007005.1,NC_034623.1,1.1
4,GCF_000007005.1,NC_034619.1,1.1
...,...,...,...
168,GCF_903886475.1,NC_002185.1,1.1
169,GCF_903886475.1,NC_004303.1,1.1
170,GCF_903886475.1,NC_031069.1,1.1
171,GCF_903886475.1,NC_022776.1,1.1


In [10]:
import numpy as np
# task1： 获取每个virus-host 的 prophage score，对于每个virus找把分数最高的host当做是病毒的host
""" 不重复
df_only=df_score.sort_values(by='score', ascending=False).groupby(
    'virus', as_index=False).first()
df_only
"""
"""重复
"""
df_score["rank"] = df_score.groupby("virus")["score"].rank(
    method="min", ascending=False).astype(np.int64)
df_only=df_score[df_score["rank"] == 1][["virus", "host","score"]]
df_only

,virus,host,score
0,NC_023585.1,GCF_000007005.1,2.3
1,NC_034620.1,GCF_000007005.1,1.4
3,NC_034623.1,GCF_000007005.1,1.1
4,NC_034619.1,GCF_000007005.1,1.1
5,NC_034628.1,GCF_000007005.1,1.1
...,...,...,...
168,NC_002185.1,GCF_903886475.1,1.1
169,NC_004303.1,GCF_903886475.1,1.1
170,NC_031069.1,GCF_903886475.1,1.1
171,NC_022776.1,GCF_903886475.1,1.1


In [11]:
# task2： 给每个virus-host 分配标签，1代表正确，0代表错误
import json

host_lineage_dict = json.load(
    open(f'/mnt/raid7/Dachuang/Achuan/03_phage_host/db/gcf_dict.json'))
virus_host_dict = json.load(
    open(f'/mnt/raid7/Dachuang/Achuan/03_phage_host/db/gold_virus_info.json'))

def species_tag(df):
    host_id = df['host']
    virus_id = df['virus'].split('.')[0]
    host_taxonomy = host_lineage_dict[host_id].split(';')[6].split('__')[1]
    virus_taxonomy_list = virus_host_dict[virus_id]['split_lineage']["species"]
    if host_taxonomy in virus_taxonomy_list:
        return 1
    else:
        return 0


def genus_tag(df):
    host_id = df['host']
    virus_id = df['virus'].split('.')[0]
    host_taxonomy = host_lineage_dict[host_id].split(';')[5].split('__')[1]
    virus_taxonomy_list = virus_host_dict[virus_id]['split_lineage']["genus"]
    if host_taxonomy in virus_taxonomy_list:
        return 1
    else:
        return 0


In [12]:

# 给df_score 分配标签，1代表正确，0代表错误
df_score['species_tag'] = df_score.apply(lambda x: species_tag(x), axis=1)
df_score['genus_tag'] = df_score.apply(lambda x: genus_tag(x), axis=1)
df_score.to_csv(
    f'{score_dir}/all_score_filtered_tag.tsv', sep='\t', index=False)


In [13]:

# 给df_only 分配标签，1代表正确，0代表错误
df_only['species_tag'] = df_only.apply(lambda x: species_tag(x), axis=1)
df_only['genus_tag']=df_only.apply(lambda x:genus_tag(x),axis=1)
df_only.to_csv(
    f'{score_dir}/only_score_filtered_tag.tsv', sep='\t', index=False)


In [14]:
df_only

,virus,host,score,species_tag,genus_tag
0,NC_023585.1,GCF_000007005.1,2.3,1,1
1,NC_034620.1,GCF_000007005.1,1.4,0,0
3,NC_034623.1,GCF_000007005.1,1.1,0,0
4,NC_034619.1,GCF_000007005.1,1.1,0,0
5,NC_034628.1,GCF_000007005.1,1.1,0,0
...,...,...,...,...,...
168,NC_002185.1,GCF_903886475.1,1.1,1,1
169,NC_004303.1,GCF_903886475.1,1.1,1,1
170,NC_031069.1,GCF_903886475.1,1.1,1,1
171,NC_022776.1,GCF_903886475.1,1.1,1,1


In [15]:
from utils.deal import *
print("all")


summary(df_score)


all


,病毒数目,宿主数目,总样本,正样本,负样本,正确率,找到正确宿主的病毒,病毒Recover-in,病毒Recover-all
species,145,73,173,120,53,0.6936,108,0.7448,0.0251
genus,145,73,173,139,34,0.8035,119,0.8207,0.0277


In [16]:
print("only")

summary(df_only)


only


,病毒数目,宿主数目,总样本,正样本,负样本,正确率,找到正确宿主的病毒,病毒Recover-in,病毒Recover-all
species,145,68,155,110,45,0.7097,107,0.7379,0.0249
genus,145,68,155,128,27,0.8258,119,0.8207,0.0277
